# Java Threads and Concurrency Utilities

## Chapter - 7
## The Locking Framework

The java.util.concurrent.locks package provides a framework of interfaces and classes for locking and waiting for conditions in a manner that’s distinct from an object’s intrinsic lock-based synchronization and java.lang.Object’s wait/notification mechanism. The concurrency utilities include the Locking Framework that improves on intrinsic synchronization and wait/notification by offering lock polling, timed waits, and more.

The Locking Framework includes the often-used Lock, ReentrantLock, Condition, ReadWriteLock, and ReentrantReadWriteLock types, which I explore in this chapter. I also briefly introduce you to the StampedLock class, which was introduced by Java 8.

## Lock
The Lock interface offers more extensive locking operations than can be obtained via the locks associated with monitors. For example, you can immediately back out of a lock-acquisition attempt when a lock isn't available. This interface declares the following methods:
* __void lock():__ Acquire the lock. When the lock isn’t available, the calling thread is forced to wait until it becomes available.
* __void lockInterruptibly():__ Acquire the lock unless the calling thread is interrupted. When the lock isn’t available, the calling thread is forced to wait until it becomes available or the thread is interrupted, which results in this method throwing java.lang. InterruptedException.
* __Condition newCondition():__ Return a new Condition instance that’s bound to this Lock instance. This method throws java.lang.UnsupportedOperationException when the Lock implementation class doesn’t support conditions.
* __boolean tryLock():__ Acquire the lock when it’s available at the time this method is invoked. The method returns true when the lock is acquired and false when the lock isn’t acquired.
* __boolean tryLock(long time, TimeUnit unit):__ Acquire the lock when it’s available within the specified waiting time, measured in unit java.util.concurrent.TimeUnit units (seconds, milliseconds, and so on), and the calling thread isn’t interrupted. When the lock isn’t available, the calling thread is forced to wait until it becomes available within the waiting time or the thread is interrupted, which results in this method throwing InterruptedException. When the lock is acquired, true is returned; otherwise, false returns.
* __void unlock():__ Release the lock.

Acquired locks must be released. In the context of synchronized methods and blocks and the implicit monitor lock associated with every object, all lock acquisition and release occurs in a block-structured manner. When multiple locks are acquired, they’re released in the opposite order and all locks are released in the same lexical scope in which they were acquired.

With this increased flexibility comes additional responsibility. The absence of block-structured locking removes the automatic release of locks that occurs with synchronized methods and blocks. As a result, you should typically employ the following idiom for lock acquisition and release:

<pre class="brush:java">
<b>
Lock l = ...; // ... is a placeholder for code that obtains the lock l.lock();
try
{
  // access the resource protected by this lock
}
catch (Exception ex)
{
  // restore invariants
}
finally {
l.unlock(); }
</b>
</pre>

__This idiom ensures that an acquired lock will always be released.__

## ReentrantLock

Lock is implemented by the ReentrantLock class, which describes a reentrant mutual exclusion lock. This lock is associated with a hold count. When a thread holds the lock and reacquires the lock by invoking lock(), lockUninterruptibly(), or one of the tryLock() methods, the hold count is increased by 1. When the thread invokes unlock(), the hold count is decremented by 1. The lock is released when this count reaches 0.

ReentrantLock offers the same concurrency and memory semantics as the implicit monitor lock that’s accessed via synchronized methods and blocks. However, it has extended capabilities and offers better performance under high thread contention (threads frequently asking to acquire a lock that’s already held by another thread). When many threads attempt to access a shared resource, the JVM spends less time scheduling these threads and more time executing them.

In [1]:
class Worker implements Runnable {
    private final String name;

    Worker(String name) {
        this.name = name;
    }

    @Override
    public void run() {
        lock.lock();
        try {
            if (lock.isHeldByCurrentThread())
                System.out.printf("Thread %s entered critical section.%n", name);
            System.out.printf("Thread %s performing work.%n", name);
            try {
                Thread.sleep(2000);
            } catch (InterruptedException ie) {
            }
            System.out.printf("Thread %s finished working.%n", name);
        } finally {
            lock.unlock();
        }
    }
}

In [2]:
import java.util.concurrent.Executors;
import java.util.concurrent.ExecutorService;
import java.util.concurrent.TimeUnit;
import java.util.concurrent.locks.ReentrantLock;

ExecutorService executor = Executors.newFixedThreadPool(2);
ReentrantLock lock = new ReentrantLock();

In [3]:
executor.execute(new Worker("ThdA"));
executor.execute(new Worker("ThdB"));
try {
    executor.awaitTermination(5, TimeUnit.SECONDS);
} catch (InterruptedException ie) {
    ie.printStackTrace();
}
executor.shutdownNow();

Thread ThdA entered critical section.
Thread ThdA performing work.
Thread ThdA finished working.
Thread ThdB entered critical section.
Thread ThdB performing work.
Thread ThdB finished working.


[]

## Condition

The Condition interface factors out Object’s wait and notification methods (wait(), notify(), and notifyAll()) into distinct condition objects to give the effect of having multiple wait-sets per object, by combining them with the use of arbitrary Lock implementations. Where Lock replaces synchronized methods and blocks, Condition replaces Object’s wait/notification methods.

A Condition instance is intrinsically bound to a lock. To obtain a Condition instance for a certain Lock instance, use Lock’s newCondition() method.

Condition declares the following methods:
* __void await():__ Force the calling thread to wait until it’s signaled or interrupted.
* __boolean await(long time, TimeUnit unit):__ Force the calling thread to wait until it’s signaled or interrupted, or until the specified waiting time elapses.
* __long awaitNanos(long nanosTimeout):__ Force the current thread to wait until it’s signaled or interrupted, or until the specified waiting time elapses.
* __void awaitUninterruptibly():__ Force the current thread to wait until it’s signaled.
* __boolean awaitUntil(Date deadline):__ Force the current thread to wait until it’s signaled or interrupted, or until the specified deadline elapses.
* __void signal():__ Wake up one waiting thread.
* __void signalAll():__ Wake up all waiting threads.

In [4]:
import java.util.concurrent.locks.Condition;
import java.util.concurrent.locks.Lock;
import java.util.concurrent.locks.ReentrantLock;

In [5]:
class Shared {
    private char c;
    private volatile boolean available;
    private final Lock lock;
    private final Condition condition;

    Shared() {
        available = false;
        lock = new ReentrantLock();
        condition = lock.newCondition();
    }

    Lock getLock() {
        return lock;
    }

    char getSharedChar() {
        lock.lock();
        try {
            while (!available)
                try {
                    condition.await();
                } catch (InterruptedException ie) {
                    ie.printStackTrace();
                }
            available = false;
            condition.signal();
        } finally {
            lock.unlock();
            return c;
        }
    }

    void setSharedChar(char c) {
        lock.lock();
        try {
            while (available)
                try {
                    condition.await();
                } catch (InterruptedException ie) {
                    ie.printStackTrace();
                }
            this.c = c;
            available = true;
            condition.signal();
        } finally {
            lock.unlock();
        }
    }
}

In [6]:
class Producer extends Thread {
    private final Lock l;

    private final Shared s;

    Producer(Shared s) {
        this.s = s;
        l = s.getLock();
    }

    @Override
    public void run() {
        for (char ch = 'A'; ch <= 'Z'; ch++) {
            l.lock();
            s.setSharedChar(ch);
            System.out.println(ch + " produced by producer.");
            l.unlock();
        }
    }
}

In [7]:
class Consumer extends Thread {
    private final Lock l;

    private final Shared s;

    Consumer(Shared s) {
        this.s = s;
        l = s.getLock();
    }

    @Override
    public void run() {
        char ch;
        do {
            l.lock();
            ch = s.getSharedChar();
            System.out.println(ch + " consumed by consumer.");
            l.unlock();
        } while (ch != 'Z');
    }
}

In [10]:
Shared s = new Shared();
Producer producer = new Producer(s);
Consumer consumer = new Consumer(s);
producer.start();
consumer.start();

try {
    producer.join();
    consumer.join();
} catch (InterruptedException e) {
}

A produced by producer.
A consumed by consumer.
B produced by producer.
B consumed by consumer.
C produced by producer.
C consumed by consumer.
D produced by producer.
D consumed by consumer.
E produced by producer.
E consumed by consumer.
F produced by producer.
F consumed by consumer.
G produced by producer.
G consumed by consumer.
H produced by producer.
H consumed by consumer.
I produced by producer.
I consumed by consumer.
J produced by producer.
J consumed by consumer.
K produced by producer.
K consumed by consumer.
L produced by producer.
L consumed by consumer.
M produced by producer.
M consumed by consumer.
N produced by producer.
N consumed by consumer.
O produced by producer.
O consumed by consumer.
P produced by producer.
P consumed by consumer.
Q produced by producer.
Q consumed by consumer.
R produced by producer.
R consumed by consumer.
S produced by producer.
S consumed by consumer.
T produced by producer.
T consumed by consumer.
U produced by producer.
U consumed by co

Listing 7-2 is similar to Listing 3-2’s PC application. However, it replaces synchronized and wait/notification with locks and conditions.
PC’s main() method instantiates the Shared, Producer, and Consumer classes. The Shared instance is passed to the Producer and

## ReadWriteLock

Situations arise where data structures are read more often than they’re modified. For example, you might have created an online dictionary of word definitions that many threads will read concurrently, while a single thread might occasionally add new definitions or update existing definitions. The Locking Framework provides a read- write locking mechanism for these situations that yields greater concurrency when reading and the safety of exclusive access when writing. This mechanism is based on the ReadWriteLock interface.

ReadWriteLock maintains a pair of locks: one lock for read-only operations and one lock for write operations. The read lock may be held simultaneously by multiple reader threads as long as there are no writers. The write lock is exclusive: only a single thread can modify shared data. (The lock that’s associated with the synchronized keyword is also exclusive.)

ReadWriteLock declares the following methods:
* Lock readLock(): Return the lock that’s used for reading.
* Lock writeLock(): Return the lock that’s used for writing.

## ReentrantReadWriteLock

ReadWriteLock is implemented by the ReentrantReadWriteLock class, which describes a reentrant read-write lock with similar semantics to ReentrantLock.

You initialize a ReentrantReadWriteLock instance by invoking either of the following constructors:
* ReentrantReadWriteLock(): Create an instance of ReentrantReadWriteLock. This constructor is equivalent to ReentrantReadWriteLock(false).
* ReentrantReadWriteLock(boolean fair): Create an instance of ReentrantReadWriteLock with the specified fairness policy. Pass true to fair when this lock should use a fair ordering policy.

In [13]:
import java.util.HashMap;
import java.util.Map;

import java.util.concurrent.Executors;
import java.util.concurrent.ExecutorService;

import java.util.concurrent.locks.Lock;
import java.util.concurrent.locks.ReadWriteLock;
import java.util.concurrent.locks.ReentrantReadWriteLock;

In [14]:
String[] words = { "hypocalcemia", "prolixity", "assiduous", "indefatigable", "castellan" };

String[] definitions = { "a deficiency of calcium in the blood", "unduly prolonged or drawn out",
        "showing great care, attention, and effort",
        "able to work or continue for a lengthy time without tiring",
        "the govenor or warden of a castle or fort" };

Map<String, String> dictionary = new HashMap<String, String>();

In [15]:
ReadWriteLock rwl = new ReentrantReadWriteLock(true);
Lock rlock = rwl.readLock();
Lock wlock = rwl.writeLock();

In [16]:
Runnable writer = () -> {
    for (int i = 0; i < words.length; i++) {
        wlock.lock();
        try {
            dictionary.put(words[i], definitions[i]);
            System.out.println("writer storing " + words[i] + " entry");
        } finally {
            wlock.unlock();
        }

        try {
            Thread.sleep(1);
        } catch (InterruptedException ie) {
            System.err.println("writer " + "interrupted");
        }
    }
};

In [17]:
Runnable reader = () -> {
    int iteration = 0;
    while (true) {
        if(iteration++ == 50)
            break;
        rlock.lock();
        try {
            int i = (int) (Math.random() * words.length);
            System.out.println("reader accessing " + words[i] + ": " + dictionary.get(words[i]) + " entry");
        } finally {
            rlock.unlock();
        }
    }
};

In [24]:
ExecutorService es1 = Executors.newFixedThreadPool(1);
es1.submit(writer);
ExecutorService es2 = Executors.newFixedThreadPool(1);
es2.submit(reader);

es1.shutdown();
es2.shutdown();

writer storing hypocalcemia entry
writer storing prolixity entry
writer storing assiduous entry
writer storing indefatigable entry
writer storing castellan entry
reader accessing prolixity: unduly prolonged or drawn out entry
reader accessing prolixity: unduly prolonged or drawn out entry
reader accessing prolixity: unduly prolonged or drawn out entry
reader accessing assiduous: showing great care, attention, and effort entry
reader accessing prolixity: unduly prolonged or drawn out entry
reader accessing castellan: the govenor or warden of a castle or fort entry
reader accessing assiduous: showing great care, attention, and effort entry
reader accessing hypocalcemia: a deficiency of calcium in the blood entry
reader accessing castellan: the govenor or warden of a castle or fort entry
reader accessing prolixity: unduly prolonged or drawn out entry
reader accessing prolixity: unduly prolonged or drawn out entry
reader accessing castellan: the govenor or warden of a castle or fort entry
